In [3]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("행성은 몇개가 있나요?")  # davinci
b = chat.predict("행성은 몇개가 있나요?") # gpt-3.5-turbo

a,b


c:\STUDY\Programing\Python\gpt\3\env\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\STUDY\Programing\Python\gpt\3\env\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('우주에는 수십억 개의 행성이 존재하는 것으로 추정됩니다. 그러나 현재까지 지구 외의 행성을 직접 관측한 것은 극희 소수뿐이기 때문에 정확한 수를 말하기는 어렵습니다.',
 '태양계에는 8개의 행성이 있습니다. 이들은 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성입니다. 추가로 소행성대 및 왜행성들도 존재합니다.')

In [6]:
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1 # 창의성을 나타내는 지표. 높을수록 창의적
)

from langchain.schema import HumanMessage, AIMessage, SystemMessage
# AIMessage : AI에 의해 보내지는거
# SystemMessage : 우리가 LLM에 설정들을 제공하기 위한 Message

messages = [
    SystemMessage(content="당신은 지리학자입니다. 당신은 오직 이탈리아어로 대답합니다."),
    # SystemMessage(content="you are a geography expert. And you only reply in italian."),
    AIMessage(content="ciao, mi chiamo Paolo"),
    HumanMessage(content="멕시코와 태국 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 무엇인가요?")
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, ma non posso fornire informazioni precise sulla distanza tra Messico e Thailandia in quanto non ho accesso a internet. Il mio nome è Paolo. Posso aiutarti con altre domande sulla geografia?')

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate : template을 message로 부터 만든다.
# PromptTemplate : string 으로 template을 만든다.

template = PromptTemplate.from_template("{국가1}과 {국가2}의 거리가 어떻게 되나요?")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1 # 창의성을 나타내는 지표. 높을수록 창의적
)

prompt = template.format(국가1='대한민국', 국가2='미국')

chat.predict(prompt)

'대한민국과 미국의 거리는 직선거리로 약 10,000km 정도입니다. 하지만 실제로 이동할 때에는 비행기나 배 등을 이용하여 이동해야 하므로 실제 이동 거리는 더 길어질 수 있습니다. 대한민국과 미국은 태평양에 위치해 있기 때문에 이동 거리가 상당히 멀다고 할 수 있습니다.'

In [4]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다. 당신은 오직 {언어}로 대답합니다."),
    ("ai", "ciao, mi chiamo {이름}"),
    ("human", "{국가1}과 {국가2}의 거리가 어떻게 되나요? 그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(
    언어 = "italian",
    이름 = "pauolo",
    국가1 = "대한민국",
    국가2 = "미국"
)

chat.predict_messages(prompt)

AIMessage(content='Mi dispiace, ma non posso fornire informazioni precise sulla distanza tra la Corea del Sud e gli Stati Uniti in questo momento. Il mio nome è Paolo. Posso aiutarti con altre domande sulla geografia?')

In [9]:
# 항상 텍스트로 대답하는 llm의 대답을 원하는 형태로 만들어보자
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, world")

['Hello', 'world']

In [19]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10, 
    question="색상이 뭐가 있을까?"
)

result = chat.predict_messages(prompt)
result

AIMessage(content='빨강, 주황, 노랑, 초록, 파랑, 남색, 보라, 핑크, 갈색, 회색')

In [20]:
p = CommaOutputParser()
p.parse(result.content)

['빨강', '주황', '노랑', '초록', '파랑', '남색', '보라', '핑크', '갈색', '회색']

In [22]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question":"포켓몬 종류 말해줘"
})

['피카츄', '파이리', '꼬부기', '이상해씨', '꼬마돌']